In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import folium                              #for maps

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
folder='Crime_Boston'
file='crimes_in_boston.csv'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Crime_Boston/crimes_in_boston.csv')# To display the top 5 rows 
df.head(5)

In [ ]:
df.dtypes

Podemos ver que el Lat y Long tienen 19.998 registros menos que Location. 
Shooting es un true unicamente cuando se registro un tiroteo dentro del crimen, a pesar de que sean pocas, es importante considerarlo.

In [ ]:
df.count()

Eliminar duplicados

In [ ]:
df.shape

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

Esto quiere decir que de las 319073 filas hay 23 duplicadas que eliminaremos.

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
print(df.isnull().sum())

1. Los que no tienen Lat y Long, vemos que es porque se seteo una Location de (0.00000000, 0.00000000). Lo podemos identificar dado que los 19998 que no tenian Lat y Long es igual a la frecuencia de (0.00000000, 0.00000000) en el campo Location.

2. Dada la repeticion del Incident number, siendo "I162030584" el mas repetido, 13 veces, podemos concluir que cada incidente puede tener mas de un registro, indicando distintos OFFENSE_CODE/GROUP/DESCRIPTION. Realizado para agregar detalle a ese incidente.

3. B2 es el distrito con mayor cantidad de registros, sin embargo, al repetirse el Incident Number, tendriamos que eliminar los repetidos para saber cual fue el número exacto de incidentes.

In [ ]:
summary = df.describe(include='all').transpose()
summary

1. Eliminamos la columna OFFENSE_CODE: Es un codigo interno de la policia de Boston, que son agrupados en un mismo OFFENSE_CODE_GROUP, que a nuestro analisis lo enriquece mucho mas, dado que brinda detalle de cuál es el tipo de Incidente. Podríamos buscar analizar los distintos offense code dentro de un mismo code group, pero como no lo vamos a hacer, la columna solo ensuciaría el análisis.

2. En nuestro análisis, creemos que la OFFENSE_DESCRIPTION, no agrega valor. La información principal a analizar se encuentra en el OFFENSE_GROUP.

3. La columna OCCURRED_ON_DATE es la agrupación de las columnas YEAR,	MONTH y	HOUR. Al separar la fecha en esos 3 campos, se facilita el analisis, por lo que eliminamos la columna OCCURRED_ON_DATE.

4. Analizaremos las ubicaciones de cada incidente, para eso, usaremos las columnas Lat y Long, que están agrupadas en la columna Location, por lo que eliminaremos está última, que no suma valor.

In [ ]:
df = df.drop(['OFFENSE_CODE', 'OFFENSE_DESCRIPTION', 'OFFENSE_DESCRIPTION', 'OCCURRED_ON_DATE', 'Location'], axis=1)
df.head(5)

Ahora para poder analizar por barrio, limpiamos los INCIDENT_NUMBER duplicados, generando un nuevo dataset

In [ ]:
duplicate_rows_df = df[df.INCIDENT_NUMBER.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
df_wo_dup_incident = df.drop_duplicates(subset=["INCIDENT_NUMBER"], keep='first')

Eliminamos los duplicados y el "dataset without duplicate incidents" es de 282517 filas y 17 columnas, a diferencia del df que es de 319073 filas y 17 columnas.

In [ ]:
df_wo_dup_incident.shape

Ahora, podemos ver que B2 sigue siendo el DISTRICT con mayores incidentes, con un número de 43403. Podemos ver que al eliminar INCIDENT_NUMBER duplicados, este número bajo desde 49940 incidentes. 

In [ ]:
summary = df_wo_dup_incident.describe(include='all').transpose()
summary

Grafico de barras de incidentes por año

In [ ]:
df_wo_dup_incident.YEAR.value_counts().nlargest(40).plot(kind='bar', figsize=(10,5))
plt.title("Número de incidentes por año")
plt.ylabel('Incidentes')
plt.xlabel('Año');

A continuación, armaremos un mapa con las longitudes y latitudes de cada incidente, que nos permitirá visualizar y comprender graficamente las ubicaciones en la ciudad. 

In [ ]:
location = df_wo_dup_incident.dropna()
location = location[["Lat", "Long", "INCIDENT_NUMBER"]]

In [ ]:
map = folium.Map(location=[location.Lat.mean(), location.Long.mean()], zoom_start=14, control_scale=True)

In [ ]:
for index, location_info in location.iterrows():
    folium.Marker([location_info["Lat"], location_info["Long"]], popup=location_info["INCIDENT_NUMBER"]).add_to(map)

In [ ]:
map